In [ ]:
from graphing.helper import convergence_analysis
from src.models.lagrangean_model import LagrangeanModel, LagrangeanAlgo
from NSPLIB.src.instances.FLECCS.src.pyomo_model.model import const_model
fleccs_obj = -4.946928843629e+08  # Replace with actual value
fleccs_sol = {'x_sorbent_total': 0.7571021770410511,'x_air_adsorb_max': 0.7567643358197031} 
fleccs_y_bound = {'x_sorbent_total': [0.0, 1.0],"x_air_adsorb_max": [0.0, 1.0]}
n_day = 7
week_diff = 52
sto_m = const_model(n_day=n_day, week_diff=week_diff)
m = LagrangeanModel.from_sto_m(sto_m)
m.build()
alg=LagrangeanAlgo(m,solver="baron",lag_iter=200)
convergence_analysis(alg,fleccs_sol,fleccs_obj,start=-4,stop=-1,steps=4,name="FLECCS")
print("FLECCS executed")

Calculating the Hausdorff distances...
	diameter = 1.0E-01, distance = 4.76E+06
